In [18]:
import ee
import geopandas as gpd
import numpy as np
import geemap

In [19]:
#Authentication
ee.Authenticate()

True

In [20]:
#Initialise
ee.Initialize()

In [21]:
#create map
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [22]:
# Load the Counties feature collection
asset_path = 'projects/ee-heat-001/assets/kenyan-counties'
kenya_counties = ee.FeatureCollection(asset_path)

In [23]:
#Filter for Mombasa county
mombasa = kenya_counties.filter(ee.Filter.eq('COUNTY', 'Mombasa'))

#Add it as a layer on the map
m.addLayer(mombasa, {}, 'Mombasa County')

In [24]:
# Load MODIS/MOD11A2
collection = ee.ImageCollection('MODIS/061/MOD11A2') \
    .filterDate('2018-01-01', '2024-06-30') \
    .filterBounds(mombasa)

# Function to transform T in Kelvin using scaling factor as provided with the link
def convertToC(image):
    result = image.multiply(0.02).subtract(273.15)
    result = result.copyProperties(image, ['system:time_start'])
    return result

collectionCelcius = collection.map(convertToC)

# Calculate mean LST in the Day
LSTmean = collectionCelcius.select('LST_Day_1km').mean()
m.addLayer(LSTmean.clip(mombasa), {
    "min": 20, "max": 40,
    "palette": ['green', 'yellow', 'darkorange', 'red']
}, 'Mean temperature Day')


In [25]:
# Calculate mean LST in the Night
LSTmean = collectionCelcius.select('LST_Night_1km').mean()

#Add it as a layer on the map
m.addLayer(LSTmean.clip(mombasa), {
    "min": 20, "max": 40,
    "palette": ['green', 'yellow', 'darkorange', 'red']
}, 'Mean temperature Night')

In [26]:
# import json

# # Get the collection info
# collection_info = collection.getInfo()

# # Pretty-print the JSON data
# print(json.dumps(collection_info, indent=2))

In [27]:
#Create another map for land surface temperature
m2 = geemap.Map()
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [28]:
#Import landsat imagery
LANDSAT/LC08/C02/T1_L2

NameError: name 'LANDSAT' is not defined